In [2]:
from keras.layers.core import Activation, Dense, Dropout
from keras.layers import concatenate,Reshape,Add,LSTM,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential,Model
from keras import Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras import Input
import keras
import numpy as np
import collections
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.preprocessing.text import text_to_word_sequence
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pymorphy2
import gensim
#nltk.download('punkt')
#nltk.download('stopwords')


Using TensorFlow backend.


In [3]:
data=np.array(pd.read_csv('data.csv'))

In [4]:
data[45]

array(['2004-01-04', 'capricorn',
       'Не будьте слишком настойчивы, побольше оригинальности в поступках. Дело, запланированное на сегодня, успешно завершится, а его плоды вы начнете собирать в скором будущем.'],
      dtype=object)

In [4]:
russian_stopwords = ['и', 'в', 'а', 'с', 'но', 'у', 'я', 'же'] #укороченная последовательность стопслов
#russian_stopwords = set(stopwords.words('russian')) #результат может ухудшаться
print(russian_stopwords)

['и', 'в', 'а', 'с', 'но', 'у', 'я', 'же']


### Функция для преобразования списка предложений в очищенный от лишних слов список предложений и разбитый по словам:

In [ ]:
def textToList(text):
review_lines = list()
for line in text:
    if line.strip() != "":
        tokens = word_tokenize(line)
        tokens = [w.lower() for w in tokens] #приводим Case к нижнему регистру
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()] #убираем токены, которые не являются символами
        words = [w for w in words if not w in russian_stopwords]
    if (len(words) > 0):
        review_lines.append(words)
return review_lines  

In [ ]:
def processText(text_list):
    tokens = [text_to_word_sequence(line, filters='!"#$%&amp;()*+,-./:;&lt;=>?@[\\]^_`{|}~\t\n\ufeff 🏠🚗😈🤦🤦‍♂️‼️♥❤️💗😻',
                                    lower=True, split=' ') for line in text_list]
    review_lines = list()
    for words in tokens:
        words = [w for w in words if w.isalpha()] #убираем токены, которые не являются символами
        words = [w for w in words if not w in russian_stopwords] #убираем стопслова
        review_lines.append(words)
    return review_lines  

In [ ]:
writers_lines = processText(writers_x_train_all)
print(writers_lines)
print(len(writers_lines))

Создадим слой embedding, обучив модель Word2Vec на своих данных.

In [ ]:
EMBEDDING_DIM = 100
w2v=gensim.models.Word2Vec(sentences = writers_lines, size = EMBEDDING_DIM, min_count = 1, window = 5)

In [ ]:
w2v = Word2Vec(min_count=1)
w2v.build_vocab(sentences)  # prepare the model vocabulary
w2v.train(sentences = writers_lines, total_examples=w2v.corpus_count, epochs=20)

In [ ]:
words = list(w2v.wv.vocab)
print("Размер словаря", len(words))